In [69]:
import random
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd
import time

MAIN_URL = 'https://www.elempleo.com/co/ofertas-empleo/ayer'
MAIN_boxes = '//div[@class="result-item"]'
XPATH_link = './/a[@class="text-ellipsis js-offer-title"]'
XPATH_name = './/a[@class="text-ellipsis js-offer-title"]'
XPATH_city = './/span[@class="info-city js-offer-city"]'
XPATH_summary = './/span[@class="text-primary info-salary js-offer-salary"]'
XPATH_company = './/span[@class="info-company-name js-offer-company"]'
XPATH_date = './/span[@class="info-publish-date pull-right js-offer-date"]'
XPATH_next_pag = '//ul[@class="pagination ee-mod "]//a[@class="js-btn-next"]'
XPATH_100 = '//select[@id="ResultsByPage"]/option[@value="100"]'
XPATH_category = './/span[@class="info-tag-not-select"]'

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
options.add_argument('--no-sandbox')
time.sleep(1)
driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)

driver.get(MAIN_URL)

button_100 = WebDriverWait(driver, 10).until(EC.presence_of_element_located \
                                         ((By.XPATH, XPATH_100)))
button.click()
time.sleep(3)

for 
    button_next = WebDriverWait(driver, 10).until(EC.presence_of_element_located \
                                             ((By.XPATH, XPATH_next_pag)))

    button_next.click()

<ipython-input-2-9384a87fc9df>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)


In [4]:
# Mostrar 100 registros
button_100 = WebDriverWait(driver, 10).until(EC.presence_of_element_located \
                                         ((By.XPATH, XPATH_100)))
button.click()

In [75]:
button_next = WebDriverWait(driver, 10).until(EC.presence_of_element_located \
                                         ((By.XPATH, XPATH_next_pag)))
button_next.click()

In [104]:
empleos = driver.find_elements_by_xpath(MAIN_boxes)

In [105]:
def join_categories(categories):
    categories_full = str()
    for category in categories:
        if categories_full:
            categories_full += ', ' + category.text
        else:
            categories_full += category.text
    return categories_full


data = pd.DataFrame(columns=['Vacante', 'Fecha', 'Compania', 'Link', 'Informacion_Adicional',
                                       'Fuente', 'Ciudad'])

for empleo in empleos:
    name =  empleo.find_element_by_xpath(XPATH_name).text
    company = empleo.find_element_by_xpath(XPATH_company).text
    salary = empleo.find_element_by_xpath(XPATH_summary).text
    city = empleo.find_element_by_xpath(XPATH_city).text
    link = empleo.find_element_by_xpath(XPATH_link).get_attribute('href')
    date = empleo.find_element_by_xpath(XPATH_date).text
    date = f'{date.split()[1]}/{datetime.today().month}/{datetime.today().year}'
    category = empleo.find_elements_by_xpath(XPATH_category)
    source = 'El Empleo'
    data = data.append({'Vacante': name, 'Fecha': date, 'Compania': company, 'Link': link, 'Salario': salary, 
                        'Informacion_Adicional': category, 'Fuente': source, 'Ciudad': city}, ignore_index=True)    
    
data

,Vacante,Fecha,Compania,Link,Informacion_Adicional,Fuente,Ciudad,Salario
0,Ejecutivo(a) comercial- medicina prepagada,15/2/2021,COOPERATIVA MEDICA DEL VALLE Y DE PROFESIONALE...,https://www.elempleo.com/co/ofertas-trabajo/ej...,[<selenium.webdriver.remote.webelement.WebElem...,El Empleo,Ibagué,"$2 a $2,5 millones"
1,Profesional investigación - análisis estadísti...,15/2/2021,CASALUKER,https://www.elempleo.com/co/ofertas-trabajo/pr...,[<selenium.webdriver.remote.webelement.WebElem...,El Empleo,Bogotá,"$2,5 a $3 millones"
2,Enfermero salud ocupacional,15/2/2021,HUMAN CONNECTION GROUP,https://www.elempleo.com/co/ofertas-trabajo/en...,[<selenium.webdriver.remote.webelement.WebElem...,El Empleo,Bogotá,"$2 a $2,5 millones"
3,Asistente de facturación - experiencia en empr...,15/2/2021,ALIANZA TEMPORALES S.A.S.,https://www.elempleo.com/co/ofertas-trabajo/as...,[<selenium.webdriver.remote.webelement.WebElem...,El Empleo,Bogotá,"$1 a $1,5 millones"
4,Profesional en medicina laboral,15/2/2021,Empresa confidencial,https://www.elempleo.com/co/ofertas-trabajo/pr...,[<selenium.webdriver.remote.webelement.WebElem...,El Empleo,Bogotá,"$2,5 a $3 millones"
...,...,...,...,...,...,...,...,...
95,Inspector de calidad,15/2/2021,Empresa confidencial,https://www.elempleo.com/co/ofertas-trabajo/in...,[<selenium.webdriver.remote.webelement.WebElem...,El Empleo,Cali,"$1 a $1,5 millones"
96,Director de mercadeo sabaneta antioquia.,15/2/2021,Empresa confidencial,https://www.elempleo.com/co/ofertas-trabajo/di...,[<selenium.webdriver.remote.webelement.WebElem...,El Empleo,Medellín,"$3 a $3,5 millones"
97,Líder calidad de software (qa),15/2/2021,InSoft SAS,https://www.elempleo.com/co/ofertas-trabajo/li...,[<selenium.webdriver.remote.webelement.WebElem...,El Empleo,Manizales,"$2 a $2,5 millones"
98,Coordinador de logistica,15/2/2021,Empresa confidencial,https://www.elempleo.com/co/ofertas-trabajo/co...,[<selenium.webdriver.remote.webelement.WebElem...,El Empleo,Toda Colombia,Salario a convenir


In [91]:
num_pags = empleos[0].find_element_by_xpath('//nav[@class="pull-right"]/span[@class="visible-lg-inline-block results-data"]').text.split()[-2]

In [93]:
int(num_pags) // 100

21

In [101]:
data.shape

(12, 8)